# Glue Elastic Views Overview

Using AWS Glue EV Console

<img src="img/img2.png">

Replicating data across multiple AWS data stores using AWS Glue Elastic Views involves the following steps:

<img src="img/img1.png">

# Step 1: Create a `DynamoDB` table (`SOURCE`)

In [ ]:
%%bash 

aws dynamodb create-table \
    --table-name favoritemovies \
    --attribute-definitions \
        AttributeName=title,AttributeType=S \
        AttributeName=release_year,AttributeType=N \
    --key-schema \
        AttributeName=title,KeyType=HASH \
        AttributeName=release_year,KeyType=RANGE \
    --provisioned-throughput \
            ReadCapacityUnits=10,WriteCapacityUnits=5 | jq . 

### Use describe-table to verify that `DynamoDB` finished creating the `favoritemovies` table and to find the `ARN` of your table

In [ ]:
!aws dynamodb describe-table --table-name favoritemovies | grep -e TableStatus -e TableArn


In [ ]:
dynamoDB_ARN = "arn:aws:dynamodb:us-east-1:507922848584:table/favoritemovies"

### Put `items` in the Table: 

In [ ]:
%%bash

# Item 1
aws dynamodb put-item \
--table-name favoritemovies  \
--item \
    '{"title": {"S": "Inception"}, "release_year": {"N": "2010"}}'

# Item 2
aws dynamodb put-item \
    --table-name favoritemovies \
    --item \
    '{"title": {"S": "Moonlight"}, "release_year": {"N": "2016"}}'

#Item 3
aws dynamodb put-item \
    --table-name favoritemovies \
    --item \
    '{"title": {"S": "The Dark Knight"}, "release_year": {"N": "2008"}}'

### Run `scan` to check that the data is added to the table 

In [ ]:
!aws dynamodb scan --table-name favoritemovies |jq . 


<img src="img/img3.png">

# Step 2: Create an Elastic View(Table and View)

### Create an Elastic Views table

In [ ]:
!aws elasticviews create-external-datastore \
    --external-datastore-identifier {dynamoDB_ARN} \
    --datastore-name evfavoritemovies --datastore-type DYNAMODB | jq . 

In [ ]:
!aws elasticviews list-sources | jq ".sources[-1]"  # Copy the sourceArn

In [ ]:
sourceArn = "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-EwCOLUylj0qLvPLYXjOCLm"

### Activate the Elastic Views table

In [ ]:
!aws elasticviews start-source \
    --source-identifier {sourceArn} | jq . 

### Check the status of the Elastic Views table

In [ ]:
!aws elasticviews describe-source \
    --source-identifier {sourceArn} | jq . 

<img src="img/img4.png">

### Create an Elastic Views view

In [ ]:
!aws elasticviews create-view \
        --view-definition 'SELECT CAST(release_year AS int4), title FROM evfavoritemovies.favoritemovies' \
        --provenances "release_year" "title" \
        --view-name favoritemovies-view | jq . 

In [ ]:
viewArn = "arn:aws:elasticviews:us-east-1:507922848584:view/favoritemovies-view-HbD9WOoulwc05EPojTJ9lm"

### Activate the Elastic Views view

In [ ]:
!aws elasticviews start-view \
    --view-identifier {viewArn} | jq . 

### Check the status of the Elastic Views view

In [ ]:
!aws elasticviews describe-view \
    --view-identifier {viewArn} | jq . 

<img src="img/img5.png">

# Step 3: Create a `Redshift` cluster (`TARGET`)

- We have already created a [Redshift Cluster](https://console.aws.amazon.com/redshiftv2/home?region=us-east-1#cluster-details?cluster=redshift-cluster-2) (Let's check via the UI console)

# Step 4: Materialization in Amazon Redshift

Lets open back to `DataGrip` and connect to our `Redshift` cluster

In [ ]:
print(viewArn) # We need this while we `CREATE MATERIALIZED VIEW`

<img src="img/img6.png">

### Put some new `item` in the Table: 

In [ ]:
!aws dynamodb put-item \
    --table-name favoritemovies  \
    --item \
        '{"title": {"S": "DDLJ"}, "release_year": {"N": "1995"}}'

### Run `scan` to check that the data is added to the table 


In [ ]:
!aws dynamodb scan --table-name favoritemovies |jq . 


# Step 5: Query Amazon `Redshift`

- Lets go back to `DataGrip` and run some query

# Cleanup

- From `Redshift` Cluster 

    - DROP MATERIALIZED VIEW favoritemovies_mv;

    - DROP SCHEMA ev_schema
    

- From `ElasticView` UI

    - Deactivate View

    - Delete View

    - Deactivate Table(source)

    - Delete Table(source)


- From `DynamoDB` UI

    - Delete the table